
# RoBERTa を 動かす
https://huggingface.co/rinna/japanese-roberta-base

- ライブラリ読み込み

In [ ]:
# Huggingface Transformersのインストール
!pip install transformers

# Sentencepieceのインストール
!pip install sentencepiece

- モデル読み込み

In [ ]:
from transformers import T5Tokenizer, RobertaForMaskedLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

- 元のテキスト入れる

In [ ]:
# original text
text_orig = "ヤマザキパンの祭りは[MASK]に開かれる。"
# text_orig = "くお～！！ぶつかる～！！ここでアクセル全開、[MASK]を右に！"
# text = "4年に1度オリンピックは開かれる。"

# prepend [CLS]
text = "[CLS]" + text_orig

# tokenize
tokens = tokenizer.tokenize(text)
print(tokens)

print('mask index :' , tokens.index('[MASK]'))
# tokens.index('オリンピック')

- mask index入れる

In [ ]:
# mask a token
masked_idx = 9 # ここにMASKのindexを入れる

In [ ]:
tokens[masked_idx] = tokenizer.mask_token
# print(tokens)  # output: ['[CLS]', '▁4', '年に', '1', '度', '[MASK]', 'は', '開かれる', '。']

# convert to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print(token_ids)  # output: [4, 1602, 44, 24, 368, 6, 11, 21583, 8]

# convert to tensor
token_tensor = torch.tensor([token_ids])


### GPU に転送するか判定

In [ ]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model = model.eval()
model = model.to(device)
token_tensor = token_tensor.to(device)

- 推論する

In [ ]:

# get the top 10 predictions of the masked token
with torch.no_grad():
    outputs = model(token_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

print(text_orig)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)